# Maximum Subarray Sum Problem 

### Problem description
Consider an array $A[1...n]$. Considering all contigious subarrays in the array $A$, find the subarray with the maximum sum over all elements. For example, consider the array $A = [13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5, -22, 15, -4, 7]$. Out of all the contigious subarrays, the subarray A[8...11] = [18, 20, -7, 12] has the maximum subarray sum of 43.

In [1]:
A = [13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5, -22, 15, -4, 7]

### Naive Solution (Bruteforce)
Just check every possible subarray

In [2]:
def naive_max_subarray(X):
    n = len(X)
    low = -1
    high = -1
    max_sum = -1
    for i in range(0, n):
        current_sum = 0
        for j in range(i, n):
            current_sum += X[j]
            if (current_sum > max_sum):
                low = i
                high = j
                max_sum = current_sum
    return (low, high, max_sum)

In [3]:
naive_max_subarray(A)

(7, 10, 43)

__Note__: python uses 0-based array indexing \
From the above nested for loop structure, its very clear that the above algorithm runs in $\mathcal{O}(n^2)$

### Divide and conquer approach
Lets say we divide the whole array into two parts, $A\lbrack1...\lfloor \frac{n}{2} \rfloor\rbrack$ and $A\lbrack(\lfloor\frac{n}{2} \rfloor+1)...n\rbrack$. We notice that the maximum contigious subarray must lie either in : 
- The left subarray 
- The right subarray 
- Crossing the mid point, ie A[i...n/2...j] 

The first two cases divide the problem into sub-problems, allowing us to use divide and conquer. The third case can be done in $\mathcal{O}(n)$ as shown below

In [4]:
import math #for infinity (just being formal)
def find_max_crossing_subarray(X, left, mid, right):
    left_max_sum = -math.inf
    left_idx = -1
    sum = 0
    for i in reversed(range(left, mid+1)): # mid...left
        sum += X[i]
        if (sum > left_max_sum):
            left_max_sum = sum
            left_idx = i
    right_max_sum = -math.inf
    right_idx = -1
    sum = 0
    for i in range(mid+1, right+1): #mid+1...right
        sum += X[i]
        if (sum > right_max_sum):
            right_max_sum = sum
            right_idx = i
    return (left_idx, right_idx, left_max_sum + right_max_sum)

In [5]:
find_max_crossing_subarray(A, 0, 7, 15)

(7, 10, 43)

Now we can use the above function to from out final divide and conquer solution as follows

In [6]:
def dnc_max_subarray(X, low, high):
    mid = (low + high)//2
    if (high == low): # Base case
        return (low, high, X[low])
    else:
        left_low, left_high, left_max_sum = dnc_max_subarray(X, low, mid)
        right_low, right_high, right_max_sum = dnc_max_subarray(X, mid+1, high)
        cross_low, cross_high, cross_max_sum = find_max_crossing_subarray(X, low, mid, high)
        if (left_max_sum >= right_max_sum) and (left_max_sum >= cross_max_sum):
            return (left_low, left_high, left_max_sum)
        elif (right_max_sum >= left_max_sum) and (right_max_sum >= cross_max_sum):
            return (right_low, right_high, right_max_sum)
        else:
            return (cross_low, cross_high, cross_max_sum)

In [7]:
dnc_max_subarray(A, 0, 15)

(7, 10, 43)

Analysing the `dnc_max_subarray(...)` function, we see that there are two call's to itself (recursively) with any input half the size of the inital call, and also a call to `find_max_crossing_subarray(...)` which we have seen takes $\mathcal{O}(n)$ time. So, we can set up the recurrence relation
$$T(n) = 2T(n/2) + \mathcal{O}(n)$$
The above recurrence relation looks like merge sorts recurrence relation, hench using master's theorem or back substitution till base case, we see that `dnc_max_subarray(...)` takes $\mathcal{O}(n\lg(n))$ time

### But wait! We can do better
A very cool insight about this problem allows us to solve in in $\mathcal{O}(n)$ time. To realize this, lets reframe the brute force solution. Rather than searching through all the contigious subarray starting from some $i$, let's look at subarrays that end with the index $i$, we notice that if we know if we know the maximum contigious subarray ending at some index $j$, say $A[m...j]$, then the max subarray ending at $j+1$ is either $[A[j+1]]$ or $A[m...j+1]$. Let's prove the above: \
Say the maximum sum subarray is not $A[m...j+1]$, but rather, say some $A[t...j+1]$ where $t \neq j+1$. That means, we are saying
$$|A[m...j+1]| \leq |A[t...j+1]| \implies |A[m...j]| + A[j+1] \leq |A[t...j]| + A[j+1] \implies |A[m...j]| \leq |A[t...j]|$$
But we assumed that $A[m...j]$ was the maximum sum subarray ending at $j$, this is a contradiction. Now the only two subarrays ending at $j+1$ is $[A[j+1]]$ and $A[m...j+1]$. Hence the maximum among them is the maximum subarray sum ending at $j+1$ $\blacksquare$ \

In [8]:
import math
def kadane_max_subarray(X):
    n = len(X)
    max_sum = -math.inf
    current_sum = -math.inf
    low, high = (-1, -1)
    current_low, current_high = (-1, -1)
    for i in range(0, n):
        current_high = i
        if (current_sum > 0): # current_sum + X[i] > X[i] => current_sum > 0
            current_sum += X[i]
        else:
            current_low = i
            current_sum = X[i]
        if (current_sum > max_sum):
            low = current_low
            high = current_high
            max_sum = current_sum
    return (low, high, max_sum)

In [9]:
kadane_max_subarray(A)

(7, 10, 43)

__This algorithm is popularly called Kadane's Algorithm__